In [1]:
import os

import cv2 as cv
from imutils import resize
import numpy as np
from matplotlib import pyplot as plt
from skimage import data

In [6]:
def nothing(x):
    pass

cap = cv.VideoCapture(0)

cv.namedWindow("Tracking")

cv.createTrackbar("LH", "Tracking", 0, 255, nothing)
cv.createTrackbar("LS", "Tracking", 0, 255, nothing)
cv.createTrackbar("LV", "Tracking", 0, 255, nothing)
cv.createTrackbar("UH", "Tracking", 255, 255, nothing)
cv.createTrackbar("US", "Tracking", 255, 255, nothing)
cv.createTrackbar("UV", "Tracking", 255, 255, nothing)

while True:
#     frame = cv.imread("./images/balloon.jpg")
    _, frame = cap.read()
    
#     frame = resize(frame, width=480)
    
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    
    l_h = cv.getTrackbarPos("LH", "Tracking")
    l_s = cv.getTrackbarPos("LS", "Tracking")
    l_v = cv.getTrackbarPos("LV", "Tracking")
    
    u_h = cv.getTrackbarPos("UH", "Tracking")
    u_s = cv.getTrackbarPos("US", "Tracking")
    u_v = cv.getTrackbarPos("UV", "Tracking")
    
    l_b = np.array([l_h, l_s, l_v])
    u_b = np.array([u_h, u_s, u_v])
    
    mask = cv.inRange(hsv, l_b, u_b)
    
    res = cv.bitwise_and(frame, frame, mask=mask)
    
    cv.imshow("frame", frame)
    cv.imshow("mask", mask)
    cv.imshow("result", res)
    
    key = cv.waitKey(1)
    if key == 27 or key == ord("q"):
        break

cap.release()
cv.destroyAllWindows()

## Smoothing and Blurring of image

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [60]:
img = cv.imread("./images/bubbles.jpg")
# img = data.chelsea
# img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img = resize(img, width = 480)

kernel = np.full((5, 5), 1, np.float32) / 25
dst = cv.filter2D(img, -1, kernel)
blur = cv.blur(img, (5, 5))
gblur = cv.GaussianBlur(img, (5, 5), 0)
mblur = cv.medianBlur(img, 5)
biFilter = cv.bilateralFilter(img, 9, 75, 75)


title = ["Image", "2D Convolution", "Blur", "Gaussian Blur", "Median Blur", "Bilateral Filter"]
images = [img, dst, blur, gblur, mblur, biFilter]


plt.figure(figsize=(10,20))
for i in range(len(title)):
    cv.imshow(title[i], images[i])

cv.waitKey(0)
cv.destroyAllWindows()

<Figure size 720x1440 with 0 Axes>

## Image pyramid

In [6]:
img = cv.imread("./images/balloon.jpg")
# img = resize(img, width=480)

l_r = cv.pyrDown(img)
h_r = cv.pyrUp(l_r)

cv.imshow("Image", img)
cv.imshow("pyrDown", l_r)
cv.imshow("pyrUP from pyrDown", h_r)
cv.waitKey(0)
cv.destroyAllWindows()

## Image Templating

In [62]:
img = cv.imread("./images/logo.png")
img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
temp = img[50:210, 300:450]

w, h = temp.shape[::-1]

res = cv.matchTemplate(img, temp, cv.TM_CCORR_NORMED)
threshold = 0.9999
loc = np.where(res >= threshold)
# print(loc)

for pt in zip(*loc[::-1]):
    cv.rectangle(img, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)

cv.imshow("image", img)
cv.imshow("template", temp)
cv.waitKey(0)
cv.destroyAllWindows()

150 90


## Background Subtraction

In [90]:
cap = cv.VideoCapture("./Videos/People Walking in the Street  _ FREE STOCK FOOTAGE - NO COPYRIGHT.mp4")

# fgbg = cv.bgsegm.createBackgroundSubtractorGMG()
fgbg = cv.createBackgroundSubtractorKNN(detectShadows = False)

# kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3,3))

while True:
    _, frame = cap.read()
    if frame is None:
        break
    
    fgMask = fgbg.apply(frame)
#     fgMask = cv.morphologyEx(fgMask, cv.MORPH_OPEN, kernel)
    
    final = cv.bitwise_and(frame, frame, mask = fgMask)
    
#     cv.imshow("Video", frame)
    cv.imshow("Masked", final)
    
    
    key = cv.waitKey(30)
    if key == 27 or key == ord('q'):
        break
        
cap.release()
cv.destroyAllWindows()